In [4]:
from src.database import IngredientsDatabase, EffectsDatabase
from src.inventory import Inventory
from src.player import Player
from src.ingredient import Ingredient
from src.alchemy_simulator import AlchemySimulator

import numpy as np
import time

In [5]:
effects_db = EffectsDatabase()
ing_db = IngredientsDatabase()

In [6]:
ing = ing_db["Small Antlers"]
print(ing)

for effect_name in ing.get_effect_names():
    effect = effects_db.ingredient_effect(effect_name, ing)
    real = effect.realize(Player())
    print(real.value)

Ingredient('Small Antlers', value=2, weight=0.1, effects=['Weakness to Poison', 'Fortify Restoration', 'Lingering Damage Stamina', 'Damage Health'])
37
80
8
2


In [11]:
# calculate effect frequencies
effects_data = {}
total = len(ing_db)

for ing in ing_db:
    effect_names = ing.get_effect_names()
    for effect_name in effect_names:
        effect = effects_db.ingredient_effect(effect_name, ing)
        realized = effect.realize(Player())
        value = realized.value
        if effect_name not in effects_data:
            effects_data[effect_name] = { 
                "count": 1,
                "total_value": value
            }
        else:
            effects_data[effect_name]["count"] += 1
            effects_data[effect_name]["total_value"] += value


            
for effect_name, data in effects_data.items():
    avg_value = data["total_value"] / data["count"]
    frequency = data["count"] / total
    utility_score = avg_value * frequency
    new_data = {"avg_value": avg_value, "frequency": frequency, "utility": utility_score}
    effects_data[effect_name] = data | new_data

max_value = max(data["avg_value"] for data in effects_data.values())
        
for effect, data in sorted(effects_data.items(), key=lambda x: x[1]["utility"]):
    print("\t" + effect + ":\n-----------------")
    print("count: " + str(data["count"]))
    print("avg value: " + str(data["avg_value"]))
    print("utility: " + str(data["utility"]))
    print("\n\n")


	Lingering Damage Stamina:
-----------------
count: 5
avg value: 8.0
utility: 0.36036036036036034



	Ravage Health:
-----------------
count: 7
avg value: 6.0
utility: 0.37837837837837834



	Cure Disease:
-----------------
count: 5
avg value: 8.6
utility: 0.38738738738738737



	Fortify Enchanting:
-----------------
count: 7
avg value: 9.0
utility: 0.5675675675675675



	Fortify Lockpicking:
-----------------
count: 5
avg value: 18.0
utility: 0.8108108108108107



	Ravage Magicka:
-----------------
count: 8
avg value: 13.25
utility: 0.954954954954955



	Fortify Barter:
-----------------
count: 4
avg value: 30.0
utility: 1.0810810810810811



	Fortify Alteration:
-----------------
count: 4
avg value: 32.0
utility: 1.1531531531531531



	Ravage Stamina:
-----------------
count: 7
avg value: 20.0
utility: 1.2612612612612613



	Weakness to Frost:
-----------------
count: 6
avg value: 26.333333333333332
utility: 1.4234234234234235



	Fortify Heavy Armor:
-----------------
count: 5
avg v

In [14]:
ingredients_data = {}

for ingredient in ing_db:
    total_value = 0
    total_utility = 0
    effect_count = 0
    for effect_name in ingredient.get_effect_names():
        effect_count += effects_data[effect_name]["count"]
        effect = effects_db.ingredient_effect(effect_name, ingredient)
        realized = effect.realize(Player())
        total_value += realized.value
        total_utility += effects_data[effect_name]["utility"]
    effect_count -= 4
    avg_utility = total_utility / 4

    ingredients_data[ingredient.name] = {
        "count": effect_count,
        "total value": total_value,
        "avg value": total_value / effect_count,
        "avg utility": avg_utility
    }

print(sorted(ingredients_data.items(), key=lambda x: x[1]["avg utility"]))

[('Scathecraw', {'count': 33, 'total value': 46, 'avg value': 1.393939393939394, 'usability value': 1518, 'avg utility': 1.1081081081081081}), ('White Cap', {'count': 32, 'total value': 96, 'avg value': 3.0, 'usability value': 3072, 'avg utility': 1.6959459459459458}), ('Hagraven Claw', {'count': 24, 'total value': 116, 'avg value': 4.833333333333333, 'usability value': 2784, 'avg utility': 1.7837837837837838}), ('Butterfly Wing', {'count': 34, 'total value': 88, 'avg value': 2.588235294117647, 'usability value': 2992, 'avg utility': 1.954954954954955}), ('Spawn Ash', {'count': 28, 'total value': 97, 'avg value': 3.4642857142857144, 'usability value': 2716, 'avg utility': 1.975225225225225}), ('Felsaad Tern Feathers', {'count': 35, 'total value': 91, 'avg value': 2.6, 'usability value': 3185, 'avg utility': 2.051801801801802}), ('Orange Dartwing', {'count': 35, 'total value': 163, 'avg value': 4.6571428571428575, 'usability value': 5705, 'avg utility': 2.4121621621621623}), ('Eye of Sa

In [7]:


inv = Inventory.generate_random_weighted(db, size=10)

for ingredient in inv.get_available_ingredients():
    print(f"{ingredient} x {inv.get_quantity(ingredient)}")

Small Antlers x 8
Garlic x 8
Netch Jelly x 1
Scaly Pholiota x 20
Slaughterfish Egg x 2
Aloe Vera Leaves x 4
Spawn Ash x 3
Bleeding Crown x 5
Briar Heart x 3
Skeever Tail x 5


In [25]:
alembic = AlchemySimulator.from_base_player()
db = IngredientsDatabase()
results = []  # Initialize results list
start_time = time.time()
for i in range(20):  # Reduced from 1M for testing
    # size = np.random.randint(3, 50)
    size = 20
    inv = Inventory.generate_normal(db, size=size)
    alembic.set_inventory(inv)
    print(f"iter {i} --- {(time.time() - start_time)} seconds --- " )
    results.append(alembic.exhaust_inventory(strategy="greedy-basic"))
    alembic.delete_inventory()

print(f"Completed {len(results)} simulations")
print(f"Average potions per run: {sum(len(r) for r in results) / len(results):.2f}")

iter 0 --- 0.042777299880981445 seconds --- 
iter 1 --- 0.3232760429382324 seconds --- 
iter 2 --- 0.6705322265625 seconds --- 
iter 3 --- 0.9385263919830322 seconds --- 
iter 4 --- 1.2303333282470703 seconds --- 
iter 5 --- 1.5927343368530273 seconds --- 
iter 6 --- 1.9833171367645264 seconds --- 
iter 7 --- 2.2575533390045166 seconds --- 
iter 8 --- 2.443692207336426 seconds --- 
iter 9 --- 2.713839292526245 seconds --- 
iter 10 --- 3.209569215774536 seconds --- 
iter 11 --- 3.597475290298462 seconds --- 
iter 12 --- 3.916405200958252 seconds --- 
iter 13 --- 4.22001838684082 seconds --- 
iter 14 --- 4.800199270248413 seconds --- 
iter 15 --- 5.316064119338989 seconds --- 
iter 16 --- 5.975863218307495 seconds --- 
iter 17 --- 6.501615285873413 seconds --- 
iter 18 --- 7.034653186798096 seconds --- 
iter 19 --- 7.296926975250244 seconds --- 
Completed 20 simulations
Average potions per run: 42.60


In [27]:
for result in results:
    print("\tinventory results:\n\t==================\n")
    for potion in result:
        
        potion.print_self()
        

	inventory results:



Poison of Damage Magicka Regen
Ingredients: Boar Tusk, Burnt Spriggan Wood, Blue Mountain Flower
Total Value: 420 gold
Number of Effects: 3

Effects:
  - Health is increased by 17 points for 60 seconds.
  - Target moves at 50% speed for 22 seconds.
  - Decrease the target's Magicka regeneration by 100% for 22 seconds.


Poison of Damage Magicka Regen
Ingredients: Ancestor Moth Wing, Chaurus Hunter Antennae, Crimson Nirnroot
Total Value: 351 gold
Number of Effects: 4

Effects:
  - Does 39 points of Stamina damage.
  - Conjuration spells last 22% longer for 60 seconds.
  - Decrease the target's Magicka regeneration by 100% for 22 seconds.
  - Items are enchanted 4% stronger for 30 seconds.


Poison of Slow
Ingredients: Deathbell, Salt Pile, Fire Salts
Total Value: 302 gold
Number of Effects: 2

Effects:
  - Target moves at 50% speed for 22 seconds.
  - Magicka regenerates 22% faster for 300 seconds.


Poison of Slow
Ingredients: Deathbell, Salt Pile, Fire Salts
Tot